# Adult black box creation

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import sys
sys.path.append("../../")

sns.set_theme()

np.random.seed(123)

In [2]:
loaded = np.load("adult-blackbox-data.npz")
print(*loaded.keys())

X_train y_train X_test y_test X_validation y_validation X_validation_noisy y_validation_noisy X_attack_2_per_quantile y_attack_2_per_quantile X_attack_3_per_quantile y_attack_3_per_quantile categorical_features numerical_features categorical_features_mask centroids


In [3]:
X_train, y_train = loaded['X_train'], loaded['y_train']
X_test, y_test = loaded['X_test'], loaded['y_test']

# Creating a Black Box

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from utils.models import save_pickle_bz2

## Random Forest

In [5]:
# Hyperparameters of the grid search
__HYPERPARAMETERS = {
    "bootstrap": [True, False],
    "max_depth": [100, 350, 500],
    "min_samples_leaf": [10, 20, 50],
    "min_samples_split": [5, 10, 50],
    "n_estimators": [100, 150, 350, 500],
    "criterion": ["gini", "entropy"],
}

def create_random_forest(
        x,
        y,
        hyperparameters=__HYPERPARAMETERS,
        n_jobs=12
) -> RandomForestClassifier:
    """Creates a random forest classifier via grid search.

    Args:
        x (ndarray): Training input examples.
        y (ndarray): Training target values.
        hyperparameters (Dict[str, List[Any]], optional): Dictionary of hyperparameters for the grid search. Defaults to the fixed ones.
        n_jobs: Number of jobs to run in parallel in the grid search.

    Returns:
        RandomForestClassifier: Random forest classifier.
    """

    rf = RandomForestClassifier()
    clf = HalvingGridSearchCV(rf, hyperparameters, refit=True, n_jobs=n_jobs, verbose=0)
    clf.fit(x, y)
    return clf.best_estimator_

In [6]:
rf = create_random_forest(X_train, y_train)

In [7]:
print(classification_report(y_test, rf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      8157
           1       0.79      0.61      0.69      2690

    accuracy                           0.86     10847
   macro avg       0.84      0.78      0.80     10847
weighted avg       0.86      0.86      0.86     10847



In [32]:
save_pickle_bz2("adult_randfor.bz2", rf)

## Adaboost

In [8]:
from sklearn.ensemble import AdaBoostClassifier

__HYPERPARAMETERS_AB = {
    "n_estimators": [50, 100, 150, 200],
    "random_state": [123]
}

def create_adaboost(
        x,
        y,
        hyperparameters=__HYPERPARAMETERS_AB,
        n_jobs=12
) -> RandomForestClassifier:
    """Creates a random forest classifier via grid search.

    Args:
        x (ndarray): Training input examples.
        y (ndarray): Training target values.
        hyperparameters (Dict[str, List[Any]], optional): Dictionary of hyperparameters for the grid search. Defaults to the fixed ones.
        n_jobs: Number of jobs to run in parallel in the grid search.

    Returns:
        RandomForestClassifier: Random forest classifier.
    """

    ab = AdaBoostClassifier()
    clf = HalvingGridSearchCV(ab, hyperparameters, refit=True, n_jobs=n_jobs, verbose=0)
    clf.fit(x, y)
    return clf.best_estimator_

In [9]:
ab = create_adaboost(X_train, y_train)

In [10]:
print(classification_report(y_test, ab.predict(X_test)))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      8157
           1       0.78      0.64      0.70      2690

    accuracy                           0.87     10847
   macro avg       0.83      0.79      0.81     10847
weighted avg       0.86      0.87      0.86     10847



In [31]:
save_pickle_bz2("adult_adab.bz2", ab)

## Neural Network

In [52]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier # this wrapper makes a keras model compatible with the sklearn APIs

def create_neural_network(input_shape, activation="relu"):
    model = keras.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Dense(32, activation=activation),
        layers.Dense(16, activation=activation),
        layers.Dense(8, activation=activation),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [53]:
nn = KerasClassifier(create_neural_network(X_train.shape[1]))

In [54]:
nn.fit(X_train, y_train, epochs=20, validation_split=0.3)

Epoch 1/20
554/554 [==============================] - 1s 1ms/step - loss: 0.4166 - accuracy: 0.8041 - val_loss: 0.3444 - val_accuracy: 0.8364
Epoch 2/20
554/554 [==============================] - 1s 1ms/step - loss: 0.3304 - accuracy: 0.8462 - val_loss: 0.3339 - val_accuracy: 0.8435
Epoch 3/20
554/554 [==============================] - 1s 1ms/step - loss: 0.3258 - accuracy: 0.8473 - val_loss: 0.3369 - val_accuracy: 0.8395
Epoch 4/20
554/554 [==============================] - 1s 1ms/step - loss: 0.3241 - accuracy: 0.8484 - val_loss: 0.3319 - val_accuracy: 0.8417
Epoch 5/20
554/554 [==============================] - 1s 1ms/step - loss: 0.3224 - accuracy: 0.8487 - val_loss: 0.3304 - val_accuracy: 0.8434
Epoch 6/20
554/554 [==============================] - 1s 1ms/step - loss: 0.3219 - accuracy: 0.8506 - val_loss: 0.3317 - val_accuracy: 0.8431
Epoch 7/20
554/554 [==============================] - 1s 1ms/step - loss: 0.3206 - accuracy: 0.8511 - val_loss: 0.3302 - val_accuracy: 0.8434
Epoch 

KerasClassifier(
	model=<keras.engine.sequential.Sequential object at 0x7f7cc3841460>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

In [55]:
print(classification_report(y_test, nn.predict(X_test)))

339/339 [==============================] - 0s 670us/step
              precision    recall  f1-score   support

           0       0.87      0.94      0.91      8157
           1       0.76      0.58      0.66      2690

    accuracy                           0.85     10847
   macro avg       0.82      0.76      0.78     10847
weighted avg       0.85      0.85      0.85     10847

